Some of the imports are not useful yet

In [1]:
from __future__ import print_function
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
import tarfile

from sklearn import preprocessing
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from six.moves import range
import tensorflow as tf
from yaafelib import *

Download GTZAN dataset once

In [12]:
def downloadGTZAN():
    filename = 'genres.tar.gz'
    if not os.path.exists(filename):
        filename, _ = urlretrieve('http://opihi.cs.uvic.ca/sound/genres.tar.gz', filename)
    else:
        print('File ' + filename + ' exists')
    
    return filename
filename = downloadGTZAN()

Extract files from .tar.gz

In [14]:
num_classes = 10

def extract(filename):
    root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
    if os.path.isdir(root):
        print('File %s already extracted' % (root))
    else:
        print('Extracting %s ' % filename)
        tar = tarfile.open(filename)
        sys.stdout.flush()
        tar.extractall()
        tar.close()
    data_folders = [os.path.join(root, d) for d in sorted(os.listdir(root))
                    if os.path.isdir(os.path.join(root, d))]
    if len(data_folders) != num_classes:
        raise Exception('Expected %d folders not found.' % (num_classes))
    print(data_folders)
    return data_folders

folders = extract(filename)

Extracting genres.tar.gz 
['genres/blues', 'genres/classical', 'genres/country', 'genres/disco', 'genres/hiphop', 'genres/jazz', 'genres/metal', 'genres/pop', 'genres/reggae', 'genres/rock']


In [ ]:
def get_features(audio_file):
    if os.path.exists(audio_file):
        print('Getting features from ' + audio_file)
    else:
        raise Exception('File ' + audio_file + ' not found')
    fp = FeaturePlan(sample_rate=22050, normalize=1)
    # Features that seems to be most often used, so they are good to start with.
    fp.addFeature('mfcc: MFCC')
    fp.addFeature('zcr: ZCR')
    fp.addFeature('spectral_shape: SpectralShapeStatistics')
    fp.addFeature('magnitude_spectrum: MagnitudeSpectrum')
    df = fp.getDataFlow()
    engine = Engine()
    engine.load(df)
    afp = AudioFileProcessor()
    afp.setOutputFormat('csv', 'features', {'Precision': '8', 'Metadata': 'False'})
    afp.processFile(engine, audio_file)
    engine.flush()
    feats = engine.readAllOutputs()
    return feats

def load_genre(folder):
    print('Loading genre from folder ' + folder)
    samples = os.listdir(folder)
    dataset =[]
    for sample in os.listdir(folder):
        sample_file = os.path.join(folder, sample)
        if sample.endswith('.au'):
            features = get_features(sample_file)
            dataset.append(features)
    return dataset

def pickle(data_folders):
    dataset_names = []
    for folder in data_folders:
        set_filename = folder + '.pickle'
        dataset_names.append(set_filename)
        if os.path.exists(set_filename):
            print('%s already pickled' % set_filename)
        else:
            print('Pickling %s.' % set_filename)
            dataset = load_genre(folder)
            with open(set_filename, 'wb') as f:
                pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
    return dataset_names

pickled_datasets = pickle(folders)

Now the data is pickled, thus the feature extraction doesn't have to be repeated. 

Let's vizualize something

In [ ]:
%matplotlib inline  

def visualize(filename, sample, feature):
    with open(filename, 'r') as f:
        unpickled = pickle.load(f)
        f = unpickled[sample][feature]
        plt.plot(f[0])
        plt.ylabel(feature)
        plt.show()

visualize('genres/rock.pickle', 0, 'magnitude_spectrum')
visualize('genres/rock.pickle', 1, 'magnitude_spectrum')
visualize('genres/rock.pickle', 2, 'magnitude_spectrum')
visualize('genres/rock.pickle', 3, 'magnitude_spectrum')
visualize('genres/blues.pickle', 0, 'magnitude_spectrum')
visualize('genres/blues.pickle', 1, 'magnitude_spectrum')
visualize('genres/blues.pickle', 2, 'magnitude_spectrum')
visualize('genres/blues.pickle', 3, 'magnitude_spectrum')

Training set and test set are needed. In addition, they have to be shuffeled/randomized somehow.

In [ ]:
pickled_datasets = ['blues.pickle','classical.pickle','country.pickle','disco.pickle','hiphop.pickle','jazz.pickle',
                    'metal.pickle','pop.pickle','reggae.pickle','rock.pickle']

def unpickle(filename):
    with open('genres/' + filename, 'r') as f:
        return pickle.load(f)
    
def evens(dataset):
    return dataset[::2]
def odds(dataset):
    return dataset[1::2]

def dataset_with_labels(filename):
    dataset = unpickle(filename)
    return dataset, [os.path.splitext(filename)[0] for count in xrange(len(dataset))]

datasets = map(dataset_with_labels, pickled_datasets)
features = [features for genre in datasets for features in genre[0]]
labels = [label for genre in datasets for label in genre[1]]

train_dataset = odds(features)
train_labels = odds(labels)
test_dataset = evens(features)
test_labels = evens(labels)

def randomize(dataset, labels):
    permutation = np.random.permutation(len(labels))
    shuffled_dataset = np.asarray(dataset)[permutation]
    shuffled_labels = np.asarray(labels)[permutation]
    return shuffled_dataset, shuffled_labels

train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)

try:
    f = open('train_and_test_data.pickle', 'wb')
    save = {
        'train_features': train_dataset,
        'train_labels': train_labels,
        'test_features': test_dataset,
        'test_labels': test_labels,
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

Now, we can go to TensorFlow

Here you can start with
=====

In [ ]:
with open('train_and_test_data.pickle', 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_features']
    train_labels = save['train_labels']
    test_dataset = save['test_features']
    test_labels = save['test_labels']
    del save  # gc
train_dataset = ([d['magnitude_spectrum'][0] for d in train_dataset])
test_dataset = ([d['magnitude_spectrum'][0] for d in test_dataset])
magnitude = 513
num_labels = 10 # genres

import numpy as np

def oneHotEncoder(pos, max):
    encoded = []
    for i in range(0, max):
        if i == pos:
            encoded.append(1)
        else:
            encoded.append(0)
    return encoded

def reformat(dataset, labels):
    dataset = np.asarray(dataset).reshape((-1, magnitude, 1, 1)).astype(np.float32)
    genres = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']
    labels = map(lambda x: np.int32(genres.index(x)), labels)
    labels = map(lambda x: oneHotEncoder(x, num_labels), labels)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

In [83]:
# Params
learning_rate = 0.001
training_iters = 400 # accuracy started decreasing for larger numbers of iterations
batch_size = 4
display_step = 10

# Network Params
n_input = 513
n_classes = 10 # genres 

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input, None, None])
y = tf.placeholder(tf.float32, [None, n_classes])

def conv2d(features, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(features, w, strides=[1, 1, 1, 1], padding='SAME'),b))

def max_pool(features, k):
    return tf.nn.max_pool(features, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(_X, _weights, _biases):
    _X = tf.reshape(_X, shape=[-1,513,1,1])

    conv1 = conv2d(_X, _weights['wc1'], _biases['bc1'])
    conv1 = max_pool(conv1, k=1)

    conv2 = conv2d(conv1, _weights['wc2'], _biases['bc2'])
    conv2 = max_pool(conv2, k=1)

    dense1 = tf.reshape(conv2, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, _weights['wd1']), _biases['bd1']))
    
    out = tf.add(tf.matmul(dense1, _weights['out']), _biases['out'])
    return out

There is a huge field for improvements here:
* Starting points should not be randomized

In [90]:
weights = {
    # 5x1 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 1, 1, 32])), 
    # 5x1 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 1, 32, 64])), 
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([64 * 513, 1024])), 
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes])) 
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
pred = conv_net(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.initialize_all_variables()

Training & Testing:
=========================

In [91]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:        
        sess.run(optimizer, feed_dict={x: train_dataset, y: train_labels})
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict={x: train_dataset, y: train_labels})
            loss = sess.run(cost, feed_dict={x: train_dataset, y: train_labels})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + \
            ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_dataset, y: test_labels}))

Iter 40, Minibatch Loss= 12503.488281, Training Accuracy= 0.36000
Iter 80, Minibatch Loss= 4351.641113, Training Accuracy= 0.54800
Iter 120, Minibatch Loss= 1680.632446, Training Accuracy= 0.69200
Iter 160, Minibatch Loss= 772.510498, Training Accuracy= 0.78000
Iter 200, Minibatch Loss= 384.756317, Training Accuracy= 0.86600
Iter 240, Minibatch Loss= 179.520569, Training Accuracy= 0.90200
Iter 280, Minibatch Loss= 88.095207, Training Accuracy= 0.93200
Iter 320, Minibatch Loss= 23.648977, Training Accuracy= 0.96800
Iter 360, Minibatch Loss= 5.767161, Training Accuracy= 0.97400
Testing Accuracy: 0.24


Classifier is disastrous. Has to be enhanced, but it's somehow working...